In [2]:
import os
class ShoppingMall:
    def __init__(self):
        self.curUid = ''
        self.curName = ''
        self.curContact = ''
        self.curEmail = ''
    def foundOwner(self, email): #working
        with open('Owners.txt','r') as file:
            line = file.readline()
            while line:
                words = line.split()
                if words[3]==email:
                    return True
                line = file.readline()
            return False
    def foundWorker(self, email): #working
        with open('Workers.txt','r') as file:
            line = file.readline()
            while line!='':
                words = line.split()
                if words[3]==email:
                    return True
                line = file.readline()
            return False
    def loginAsUser(self, user): #working
        while True:
            inp = input('Enter your email (enter "BACK" to go back): ')
            if inp.upper()=='BACK':
                break
            elif inp!='':
                if (user=='Worker' and self.foundWorker(inp)) or (user=='Owner' and self.foundOwner(inp)):
                    while True:
                        passInput = input('Enter your Password (enter "BACK" to go back): ')
                        if passInput.upper()=='BACK':
                            break
                        elif user=='Worker' and self.varifyWorker(inp, passInput):
                            print('Login Successfull.')
                            self.startWorkerSession()
                            return
                        elif user=='Owner' and self.varifyOwner(inp, passInput):
                            print('Login Successfull.')
                            self.startOwnerSession()
                            return
                        else:
                            print('! Incorrect Password. Enter Again!')
                else:
                    print('! Email not found. Enter again!')
            else:
                print('! Input can not be blank. Try again!')
    def addInventory(self): #working
        with open('Products.txt', 'r') as file:
            products = [x.split()[1] for x in file.readlines()]
        while True:
            search = input('Search Product (enter "BACK" to stop adding new product): ').lower()
            rawBuff = []
            if search.lower()=='back':
                break
            search = search.split()
            rcount = 0
            for i in products:
                c = len(search)
                count = 0
                for j in search:
                    if j in i.lower():
                        count += 1
                if count>=c:
                    rawBuff.append(i)
            if len(rawBuff) == 0:
                print('! No Product found Search again!')
                    
            elif len(rawBuff) == 1:
                details = self.getProductDetails(rawBuff[0])
                print('Selected Product:',' '.join(rawBuff[0].split('_')))
                while True:
                    inpquant = input('Enter the quantity: ')
                    if inpquant.isnumeric() and int(inpquant)>=0:
                        self.addQuantity(details[0], inpquant)
                        break
                    else:
                        print('! Invalid Quantity. Enter again!')
            else:
                print('Results:')
                print('Enter Unique name from below list.')
                for i in rawBuff:
                    print(' '.join(i.split('_')))
    def addQuantity(self, pid, quantity): #working
        with open('Products.txt', 'r') as file:
            read = file.readlines()
        with open('Products.txt', 'w') as file:
            for i in read:
                words = i.split()
                if words[0]==pid:
                    words[3] = str(int(words[3]) + int(quantity))
                    file.write(' '.join(words)+' \n')
                else:
                    file.write(i)
    def varifyWorker(self, email, uPass): #working
        with open('Workers.txt','r') as file:
            line = file.readline()
            while line!='':
                words = line.split()
                if words[3]==email and words[5]==uPass:
                    self.curUid, self.curName, self.curEmail, self.curContact = words[0],(words[1]+' '+words[2]), words[3], words[4]
                    return True
                line = file.readline()
            return False
    
    def varifyOwner(self, email, uPass): #working
        with open('Owners.txt','r') as file:
            line = file.readline()
            while line!='':
                words = line.split()
                if words[3]==email and words[5]==uPass:
                    self.curUid, self.curName, self.curEmail, self.curContact = words[0],(words[1]+' '+words[2]), words[3], words[4]
                    return True
                line = file.readline()
            return False
    def getBillNo(self): #working
        listBills = os.listdir('Bills\\')
        if len(listBills)>0:
            lastBillName = listBills[-1]
            with open('Bills\\' + lastBillName, 'r') as file:
                lastBillNo = int(file.readline().split()[-1])
            return lastBillNo+1
        else:
            return 1
        
    def createBill(self): #working
        billNo = str(self.getBillNo())
        billString = 'Bill No: ' + billNo + ' \nCustomer Name: '
        billProduct = []
        total = 0
        p_no = 1
        while True:
            custName = input('Enter Customer Name (enter "BACK" to go back): ')
            if custName.lower()=='back':
                return
            else:
                break
        while True:
            custContact = input('Enter Customer Phone no.: ')
            if len(custContact)==10 and custContact.isnumeric():
                break
            else:
                print('! Invalid Phone number. Enter Again!')
        billString += custName + ' Phone No.: '+ custContact +'\n\nProducts:\n'
        with open('Products.txt', 'r') as file:
            products = [x.split()[1] for x in file.readlines()]
        while True:
            search = input('Search Product (enter "BACK" to stop adding new product): ').lower()
            rawBuff = []
            if search.lower()=='back':
                if len(billProduct)>0:
                    for i in billProduct:
                        billString += i + '\n'
                    billString += 'Total: ' + str(total)
                    self.generateBill(billNo, billString, total)
                break
            search = search.split()
            rcount = 0
            for i in products:
                c = len(search)
                count = 0
                for j in search:
                    if j in i.lower():
                        count+= 1
                if count>=c:
                    rawBuff.append(i)
            if len(rawBuff) ==0:
                print('! No Product found Search again!')
            elif len(rawBuff) == 1:
                details = self.getProductDetails(rawBuff[0])
                quantity = int(details[3])
                print('Product Selected:', ' '.join(rawBuff[0].split('_')))
                while True:
                    inpquant = input('Enter the quantity: ')
                    if inpquant.isnumeric():
                        if int(inpquant)>0 and int(inpquant)<= quantity:
                            self.removeQuantity(details[0], inpquant)
                            billEntryString = str(p_no) + ' ' + ' '.join(details[1].split('_')) + ' Rate: ' + details[2] + ' Quantity: ' + inpquant
                            discount = float(details[4])
                            if discount !=0:
                                price = (float(details[2]) - discount*float(details[2])/100) * int(inpquant)
                                
                                billEntryString += ' Subtotal: ' + str(price) + ' ( ' + str(discount) + '% Discount applied.)'
                            else:
                                price = float(details[2]) * int(inpquant)
                                billEntryString += ' Subtotal: ' + str(price)
                            total += price
                            billProduct.append(billEntryString)
                            p_no += 1
                            print('Product Added.')
                            break
                        else:
                            print('! Insufficient Stock. ', end='')
                            if int(quantity)>0:
                                print('Only', quantity, 'available!')
                            else:
                                print('No stock available!')
                                break
                    else:
                        print('! Invalid Quantity. Enter again!')
            else:
                print('Results:')
                print('Enter Unique name from below list.')
                for i in rawBuff:
                    print(' '.join(i.split('_')))
            
    def generateBill(self, billNo, content, totalAmount): #working
        with open('Bills\\'+billNo+'.txt', 'w') as file:
            file.write(content)
        with open('Private\\stats.txt', 'r') as file:
            read = file.readlines()
        line = read[2]
        lineList = line.split()
        lineList[-1] = str(int(lineList[-1]) + 1)
        line = ' '.join(lineList) + ' \n'
        read[2] = line
        
        line = read[0]
        lineList = line.split()
        lineList[-1] = str(float(lineList[-1]) + totalAmount)
        line = ' '.join(lineList) + ' \n'
        read[0] = line
        with open('Private\\stats.txt', 'w') as file:
            file.writelines(read)
        print('Bill Printed.')
    def removeQuantity(self, pid, quantity): ##working
        intQuantity = int(quantity)
        with open('Products.txt', 'r') as file:
            read = file.readlines()
            for i in range(len(read)):
                words = read[i].split()
                if words[0]==pid:
                    words[3] = str(int(words[3]) - intQuantity)
                    words[5] = str(int(words[5]) + intQuantity)
                    read[i] = ' '.join(words) + ' \n'
        with open('Products.txt', 'w') as file:
            file.writelines(read)
        with open('Private\\stats.txt', 'r') as file:
            read = file.readlines()
        line = read[1]
        lineList = line.split()
        lineList[-1] = str(int(lineList[-1]) + intQuantity)
        
        line = ' '.join(lineList) + ' \n'
        read[1] = line
        with open('Private\\stats.txt', 'w') as file:
            file.writelines(read)
    def getProductDetails(self, searchName): #working
        with open('Products.txt', 'r') as file:
            ls = file.readlines()
        for x in ls:
            details = x.split()
            if details[1] == searchName:
                return details
        print('No match found.')
        return []
    def addProduct(self):
        with open('Products.txt', 'r') as file:
            lines = file.readlines()
        newId = str(int(lines[-1].split()[0]) + 1)
        while True:
            pName = input('Enter Product Name (enter "BACK" to go back): ')
            if pName.lower() == 'back':
                break
            else:
                pName = '_'.join(pName.split())
            while True:
                pPrice = input('Enter Product Price: ')
                if pPrice.isnumeric():
                    break
                else:
                    print('! Invalid Input. Try again!')
            while True:
                stock = input('Enter Stock: ')
                if stock.isnumeric():
                    break
                else:
                    print('! Invalid Input. Try again!')
            with open('Products.txt', 'a') as file:
                file.write(newId + ' ' + pName + ' ' + pPrice + ' ' + stock + ' 0 0 \n')
            print('New Product added.')
            newId = str(int(newId)+1)

    def orderNewSupply(self): #working
        string ='Supply Order:\nThreshold Quantity: '
        c = 1
        while True:
            inp = input('Enter a Quantity threshold to list low quantity stocks (Enter "BACK" to go back): ')
            if inp.lower()=='back':
                break
            elif inp.isnumeric() and int(inp)>=0:
                string+= inp+'\nProduct List:\n'
                inp = int(inp)
                with open('Products.txt','r') as file:
                    read = file.readlines()
                for i in read:
                    words = i.split()
                    if int(words[3])<=inp:
                        substr = str(c)+'. PID: '+ words[0] + ' Product: ' + ' '.join(words[1].split('_')) + ' Stock: ' + words[3] +'\n'
                        print(substr,end='')
                        c+=1
                        string+= substr
                with open('Private\\SupplyOrder.txt', 'w') as file:
                    file.write(string)
                    print('Supply Order Generated.')
                
            else:
                print('!Invalid input. Try again!')
        
        
    def addNewWorker(self): #working
        with open('Workers.txt', 'r') as file:
            lines = file.readlines()
        newId = str(int(lines[-1].split()[0]) + 1)
        while True:
            firstName = input('Enter First Name (enter "BACK" to go back): ')
            if firstName=='':
                print('! First Name can not be blank!')
            elif firstName.lower() == 'back':
                return
            else:
                break
        while True:
            lastName = input('Enter Last Name (enter "BACK" to go back): ')
            if lastName=='':
                print('! Last Name can not be blank. Try again!')
            else:
                break
        while True:
            workerEmail = input("Enter worker's Email: ")
            if workerEmail=='':
                print('! Email can not be blank. Try again!')
            else:
                break
        while True:
            workerContact = input("Enter worker's Contact Number: ")
            if len(workerContact)==10 and workerContact.isnumeric():
                break
            else:
                print('! Invalid contact. Enter again!')
        while True:
            workerPass = input("Enter worker's Password: ")
            if workerPass=='':
                print('! Password can not be blank. Try again!')
            else:
                break
        workerString = ' '.join([newId, firstName, lastName, workerEmail, workerContact, workerPass, '\n'])
        with open('Workers.txt', 'a') as file:
            file.write(workerString)
        print('New Worker Added.')

                
    def removeWorker(self): #working
        while True:
            em = input('Enter an email of worker to be removed (enter "BACK" to go back): ')
            if em=='':
                print('! Email can not be empty. Enter again!')
            elif em.lower() == 'back':
                return
            else:
                break
        with open('Workers.txt', 'r') as file:
            ls = file.readlines()
        newls = []
        isFound = False
        for worker in ls:
            words = worker.split()
            if words[3]!=em:
                newls.append(worker)
            else:
                isFound = True
                print('Worker Removed:',words[1],words[2])
        if isFound:
            with open('Workers.txt', 'w') as file:
                file.writelines(newls)
        else:
            print('! Worker not found. Try again!')
                    
            
    def changeDiscount(self): #working
        with open('Products.txt', 'r') as file:
            products = [x.split()[1] for x in file.readlines()]
        while True:
            search = input('Search Product (enter "BACK" to stop changing discounts): ').lower()
            rawBuff = []
            if search.lower()=='back':
                break
            search = search.split()
            rcount = 0
            for i in products:
                c = len(search)
                count = 0
                for j in search:
                    if j in i.lower():
                        count += 1
                if count>=c:
                    rawBuff.append(i)
            if len(rawBuff) == 0:
                print('! No Product found Search again!')
                    
            elif len(rawBuff) == 1:
                details = self.getProductDetails(rawBuff[0])
                print(details[4])
                discount = float(details[4])
                print('Current discount of', ' '.join(rawBuff[0].split('_'))+':', str(discount)+'%')
                while True:
                    inpdisc = input('Enter the new discount in percentage: ')
                    try:
                        rawdata = float(inpdisc)
                    except ValueError as e:
                        print('! Invalid Discount. Enter again!')
                    else:
                        self.editDiscount(details[0], inpdisc)
                        print('Discount changed.')
                        break
            else:
                print('Results:')
                print('Enter Unique name from below list.')
                for i in rawBuff:
                    print(' '.join(i.split('_')))
    def editDiscount(self, pid, discount): #working
        with open('Products.txt', 'r') as file:
            read = file.readlines()
        with open('Products.txt', 'w') as file:
            for i in read:
                words = i.split()
                if words[0]==pid:
                    words[4] = str(float(discount))
                    file.write(' '.join(words) + ' \n')
                else:
                    file.write(i)
    def mostBought(self): #working
        with open('Products.txt','r') as file:
            read = file.readlines()
        read = sorted(read, key=lambda a:int(a.split()[5]), reverse=True)
        c = 1
        for i in read:
            words = i.split()
            print('Product:',' '.join(words[1].split('_')), ' Total Quantity Sold:', words[-1])
            if c==10:
                break
            else:
                c+=1
    def showAllWorkers(self):
        print('Workers List: ')
        with open('Workers.txt', 'r') as file:
            ls = file.readlines()
        for person in ls:
            words = person.split()
            print('UID: ', words[0],'First Name:', words[1], 'Last Name:', words[2], 'Email:', words[3], 'Contact:', words[4])
    def statistics(self):
        with open('Private\\stats.txt', 'r') as file:
            print(file.read())
    def startOwnerSession(self):
        while True:
            print('\nUser ID:',self.curUid,' Name:',self.curName,'\n1. Add new Worker\n2. Remove Worker\n3. Change Discount Offer\n4. Show top most Bought Products\n5. Show all Workers\n6. Show Shopping Mall Statistics\n7. Exit')
            inp = input('Enter Option: ')
            if inp=='1':
                self.addNewWorker()#working
            elif inp=='2':
                self.removeWorker()#working
            elif inp=='3':
                self.changeDiscount()#working
            elif inp=='4':
                self.mostBought()#working
            elif inp=='5':
                self.showAllWorkers()#working
            elif inp=='6':
                self.statistics()#working
            elif inp=='7':
                break
            else:
                print('! Invalid input. Try again!')
    def startWorkerSession(self):
        while True:
            print('\nUser ID:',self.curUid,' Name:', self.curName,'\n1. Create Bill\n2. Add new Product\n3. Order new supply\n4. Exit')
            inp = input('Enter Option: ')
            if inp=='1':
                self.createBill() #working
            elif inp=='2':
                self.addProduct() #working
            elif inp=='3':
                self.orderNewSupply() #working
            elif inp=='4':
                break
            else:
                print('! Invalid input. Try again!')
        

                    
    def start(self):
        while True:
            print('\n---------- Main Menu ----------\n1. Login as Worker\n2. Add Inventory\n3. Login as Owner\n4. Exit')
            option = input('Enter your option: ')
            if option == '1':
                self.loginAsUser('Worker')
            elif option == '2':
                self.addInventory() #working
            elif option == '3':
                self.loginAsUser('Owner')
            elif option == '4':
                print('Exitting...')
                break
            else:
                print('! Enter valid option!')

SMS = ShoppingMall()
SMS.start()


---------- Main Menu ----------
1. Login as Worker
2. Add Inventory
3. Login as Owner
4. Exit
Enter your option: 1
Enter your email (enter "BACK" to go back): -4
! Email not found. Enter again!
Enter your email (enter "BACK" to go back): dharmesh@gmail.com
! Email not found. Enter again!
Enter your email (enter "BACK" to go back): padharam@gmail.com
Enter your Password (enter "BACK" to go back): 12345
Login Successfull.

User ID: 1  Name: Dharmesh Padhra 
1. Create Bill
2. Add new Product
3. Order new supply
4. Exit
Enter Option: 1
Enter Customer Name (enter "BACK" to go back): dharmesh
Enter Customer Phone no.: 2342342345
Search Product (enter "BACK" to stop adding new product): atta
Results:
Enter Unique name from below list.
Aashirvaad Atta Whole Wheat 10kg
Aashirvaad Atta Whole Wheat 5kg
Aashirvaad Atta Whole Wheat 1kg
Aashirvaad Atta Whole Wheat 2kg
Search Product (enter "BACK" to stop adding new product): back

User ID: 1  Name: Dharmesh Padhra 
1. Create Bill
2. Add new Product